In [1]:
import tweepy
import textblob
import pandas as pd
import numpy as np
import csv

#setting up access to Twitter App
api_key = 'Zt0jMOLNAoMChLRSSFu3hiGX7'
api_secret = 'SRWSr62brD1I1SanBu01qRAppjwAFenDaSkvmbh2Fd63rJ8j9E'
token = '1859101303-I4C7lCCGFMdmhKOxQL1cdzTF14qcu1rk7kfR5li'
token_secret = 'KTB0t7w4sFddlskb7hIn9gVBa4c4xDsD8y4SZtYkvzFxt'

#logging in to Twitter
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(token, token_secret)

api = tweepy.API(auth) 

Nin_tweet_lst = [] #list to contain all Nintendo tweets
Xbox_tweet_lst = [] #list to contain all Xbox tweets
PS_tweet_lst = [] #list to contain all PlayStation tweets
Nin_analysis_lst = [] #list to contain all Nintendo tweets
Xbox_analysis_lst = [] #list to contain all Xbox tweets
PS_analysis_lst = [] #list to contain all PlayStation tweets

In [2]:
#retrive last 500 tweets containing Nintendo Switch; store them in Nintendo tweet list
for t in tweepy.Cursor(api.search, q='Nintendo Switch', lang='en').items(500):
    Nin_tweet_lst.append(t.text.encode('utf-8'))
    analysis = textblob.TextBlob(t.text)
    a = analysis.sentiment
    Nin_analysis_lst.append(a)

In [3]:
#retrive last 500 tweets containing Xbox One; store them in Xbox tweet list
for t in tweepy.Cursor(api.search, q='Xbox One', lang='en').items(500):
    Xbox_tweet_lst.append(t.text.encode('utf-8'))
    analysis = textblob.TextBlob(t.text)
    a = analysis.sentiment
    Xbox_analysis_lst.append(a)

In [4]:
#retrive last 500 tweets containing PS4; store them in PlayStation tweet list
for t in tweepy.Cursor(api.search, q='PS4', lang='en').items(500):
    PS_tweet_lst.append(t.text.encode('utf-8'))
    analysis = textblob.TextBlob(t.text)
    a = analysis.sentiment
    PS_analysis_lst.append(a)

In [5]:
#Create dataframes to house scraped data
Nintendo_DF = pd.DataFrame({
    'Tweet': Nin_tweet_lst,
    'Sentiment': Nin_analysis_lst
})

Xbox_DF = pd.DataFrame({
    'Tweet': Xbox_tweet_lst,
    'Sentiment': Xbox_analysis_lst
})

PS_DF = pd.DataFrame({
    'Tweet': PS_tweet_lst,
    'Sentiment': PS_analysis_lst
})

In [6]:
#Create separate DataFrame columns for Polarity & Subjectivity
Nintendo_DF = Nintendo_DF.join(pd.DataFrame(Nintendo_DF['Sentiment'].astype(str).str.split(',').tolist(),columns=['Polarity', 'Subjectivity']))
Xbox_DF = Xbox_DF.join(pd.DataFrame(Xbox_DF['Sentiment'].astype(str).str.split(',').tolist(),columns=['Polarity', 'Subjectivity']))
PS_DF = PS_DF.join(pd.DataFrame(PS_DF['Sentiment'].astype(str).str.split(',').tolist(),columns=['Polarity', 'Subjectivity']))

In [7]:
#Remove unnecessary text from Polarity and Subjectivity columns; add extra column with console name
Nintendo_DF['Polarity'] = Nintendo_DF['Polarity'].str.replace('=','').str.replace('(','').str.replace('Sentiment','').str.replace('polarity','')
Nintendo_DF['Subjectivity'] = Nintendo_DF['Subjectivity'].str.replace('subjectivity=','').str.replace(')','')
Nintendo_DF['Console'] = 'Nintendo Switch'

Xbox_DF['Polarity'] = Xbox_DF['Polarity'].str.replace('=','').str.replace('(','').str.replace('Sentiment','').str.replace('polarity','')
Xbox_DF['Subjectivity'] = Xbox_DF['Subjectivity'].str.replace('subjectivity=','').str.replace(')','')
Xbox_DF['Console'] = 'Xbox One'

PS_DF['Polarity'] = PS_DF['Polarity'].str.replace('=','').str.replace('(','').str.replace('Sentiment','').str.replace('polarity','')
PS_DF['Subjectivity'] = PS_DF['Subjectivity'].str.replace('subjectivity=','').str.replace(')','')
PS_DF['Console'] = 'PS4'

In [8]:
del PS_DF['Sentiment']
del Nintendo_DF['Sentiment']
del Xbox_DF['Sentiment']

In [9]:
print type(PS_DF['Tweet'][0])

<type 'str'>


In [10]:
PS_DF.to_csv('playstation_tweets.csv')
Nintendo_DF.to_csv('nintendo_tweets.csv')
Xbox_DF.to_csv('xbox_tweets.csv')

In [11]:
ConsoleSentiment_DF = Nintendo_DF.append([PS_DF, Xbox_DF])

In [13]:
ConsoleSentiment_DF.to_csv('ConsoleSentiment.csv')